## Notebook Goals

Note: this is a [unfinished] side path trying to find optimal model. This notebook is not apart of main solution

The goal of this notebook is to find the best model and hyper-parameter tuning

## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [4]:
df = pd.read_csv('../data/amzn_final_dataset.csv',index_col='AMZN')
df.dropna(inplace=True)

In [5]:
df = df.iloc[-500:]
df

,Open,High,Low,Close,Volume,r_percent_increase_high,r_percent_increase_close,c_four_percent_high,SMA,SMA_diff,...,rsi_over_60,rsi_under_40,rsi_under_30,rsi_under_20,sto_over_80,sto_over_70,sto_over_60,sto_under_40,sto_under_30,sto_under_20
AMZN,,,,,,,,,,,,,,,,,,,,,
2019-02-15,1627.86,1628.9100,1604.50,1607.95,4343893.0,2.901210,1.557884,0,1622.363333,-2.817778,...,0,0,0,0,0,0,0,1,1,1
2019-02-19,1601.00,1634.0000,1600.56,1627.58,3681656.0,1.660133,0.541909,0,1618.893333,-3.470000,...,0,0,0,0,0,0,0,1,0,0
2019-02-20,1630.00,1634.9300,1610.12,1622.10,3337589.0,2.003576,1.170705,0,1616.875556,-2.017778,...,0,0,0,0,0,0,0,1,0,0
2019-02-21,1619.85,1623.5600,1600.91,1619.44,3483392.0,2.171121,1.336882,0,1617.438889,0.563333,...,0,1,0,0,0,0,0,0,0,0
2019-02-22,1623.50,1634.9400,1621.17,1631.56,3096191.0,2.617127,2.462061,0,1622.254444,4.815556,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-03,3425.01,3434.0000,3308.62,3312.53,7088781.0,1.946247,1.196065,0,3291.574444,0.615556,...,1,0,0,0,0,0,1,0,0,0
2021-02-04,3330.00,3347.0000,3277.75,3331.00,3670661.0,1.380967,0.634944,0,3295.882222,4.307778,...,1,0,0,0,0,0,1,0,0,0
2021-02-05,3319.00,3377.0000,3302.71,3352.15,3620808.0,0.383333,-0.871381,0,3302.343333,6.461111,...,1,0,0,0,0,1,1,0,0,0


## Train/Test Split

In [7]:
y = df['r_percent_increase_close']
x = df[['SMA', 'SMA_diff', 'Stochastic', 'Stochastic_diff', 'RSI', 'RSI_diff', 'ROC', 'ROC_diff', 'ATR', 'ATR_diff', 'ADX', 'ADX_diff']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [8]:
scaler = StandardScaler()

scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.fit_transform(x_test)

## Grid Search CV

In [9]:
dtr = DecisionTreeRegressor()
svr = SVR()
knr = KNeighborsRegressor()
forest = RandomForestRegressor()
ada_reg = AdaBoostRegressor()
grad_boost_reg = GradientBoostingRegressor()

models = [dtr, svr, knr, forest, ada_reg, grad_boost_reg]

In [ ]:
for model in models:
    
    if model == dtr:
        param_grid = {
               'criterion': ['mse', 'friedman_mse', 'mae'],
               'max_depth': [None, 2, 3, 4, 5, 6],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 3, 4, 5, 6],
               'max_features': ['auto', 'sqrt', 'log2']}
        
        
    if model == svr:
        param_grid = {
                'kernel': ['linear', 'rbf'],
                'degree' : [1,3,5],
                'gamma': [0.001, 0.01, 0.1, 1],
                'C': [0.001, 0.01, 0.1, 1, 10]}
 

    if model == knr:
        param_grid = {
               'n_neighbors': [3,5,7,11,19],
               'weights': ['uniform', 'distance'],
               'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
               'metric' : ['minkowski', 'euclidean', 'manhattan']}
    
    
    if model == forest:
        param_grid = {
                'n_estimators': [20, 50, 100, 300, 500],
                'criterion': ['mse', 'mae'],
                'max_depth': [5, 8, 15, 25, 30],
                'min_samples_split': [2, 5, 10, 15, 100],
                'min_samples_leaf': [1, 2, 5, 10],
                'max_features' : ['auto', 'sqrt', 'log2']}
    
    
    if model == ada_reg:
        param_grid = {
                'n_estimators': [50, 100, 150, 200, 250],
                'learning_rate': [0.01, 0.05, 0.1,  0.2, 0.5, 1],
                'loss' : ['linear', 'square', 'exponential']}
    
        
    if model == grad_boost_reg:
        param_grid = {
                'loss' : ['ls', 'lad', 'huber', 'quantile'],
                'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5],
                'n_estimators': [25, 50, 75, 100, 150, 200, 250],
                'criterion' : ['friedman_mse', 'mse']}
                
    
    grid_search = GridSearchCV(model, param_grid, cv=3, return_train_score=True, scoring='r2') #scoring='r2'

    if model == KNeighborsRegressor() or model == SVR():
        grid_search.fit(scaled_x_train, y_train)

        #Mean training score
        gs_training_score = np.mean(grid_search.cv_results_['mean_train_score'])

        #Mean test score
        gs_testing_score = grid_search.score(scaled_x_test, y_test)

    else:
        grid_search.fit(x_train, y_train)

        #Mean training score
        gs_training_score = np.mean(grid_search.cv_results_['mean_train_score'])

        #Mean test score
        gs_testing_score = grid_search.score(x_test, y_test)

    print(model, '\n')
    print(f"Mean Training Score: {gs_training_score :.2%}")
    print(f"Mean Test Score: {gs_testing_score :.2%}")
    print("Best Parameter Combination Found During Grid Search:")
    print(grid_search.best_params_)
    print('\n\n')